**STEP 1 - DOWNLOADING THE REQ PACKAGES AND LIB**

In [ ]:
!pip install langchain-classic
!pip install chromadb openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 38.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.1 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.


In [ ]:
# the req lib
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [ ]:
# use if u want to use a dropbox doc for the input data else upload mannualy
!wget -q = "link of dropbox"
!unzip -q "path of the folder" -d "path of the folder but wihout the zip command"

**STEP 2 - SETTING UP THE OPEN AI KEY**

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ""

**STEP 3 - LOADING THE DATA**

In [ ]:
loader = DirectoryLoader("/content/new_articles/", glob = "./*.txt", loader_cls= TextLoader)
document = loader.load()

**STEP 4 - CHUNKING**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
text = text_splitter.split_documents(document)

**STEP 5 - CREATING THE DB**

In [ ]:
from langchain import embeddings
persist_directory = 'db'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=text,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# saving the db to disk and loading it from the disc for usage
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

**STEP 6 - MAKING THE RETRIVER FUNC**

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2}) # TO SET THE NUM OF OUTPUTS can also include the search type ie similarity search

**STEP 7 - MAKING QA CHAIN AS PER LANG CLASSIC**

In [ ]:
llm=OpenAI()
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
# printing the answer along with its reference sources.
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

**STEP 8 - GENERATING THE ANS**

In [ ]:
query = "How much money did Microsoft raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)